In [80]:
import numpy as np
import random
from implementations import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
LOWER_BOUND = -1
UPPER_BOUND = 1

# Import data

In [82]:
DATA_FOLDER = "competition-data/"
DATA_TEST = "test.csv"
DATA_TRAIN = "train.csv"

In [83]:
from proj1_helpers import load_csv_data
y_train, x_train, ids_train = load_csv_data(DATA_FOLDER + DATA_TRAIN, LOWER_BOUND, UPPER_BOUND)
y_test, x_test, ids_test = load_csv_data(DATA_FOLDER + DATA_TEST, LOWER_BOUND, UPPER_BOUND)

In [84]:
x_train

array([[ 138.47 ,   51.655,   97.827, ...,    1.24 ,   -2.475,  113.497],
       [ 160.937,   68.768,  103.235, ...,      nan,      nan,   46.226],
       [     nan,  162.172,  125.953, ...,      nan,      nan,   44.251],
       ..., 
       [ 105.457,   60.526,   75.839, ...,      nan,      nan,   41.992],
       [  94.951,   19.362,   68.812, ...,      nan,      nan,    0.   ],
       [     nan,   72.756,   70.831, ...,      nan,      nan,    0.   ]])

In [85]:
tx_train = x_train.T
tx_test = x_test.T

# Cleaning data

In [86]:
"""# constants
CATEGORICAL_FIELDS = ['PRI_jet_num']

for c in CATEGORICAL_FIELDS:
    median_rep_data_train[c] = categorical_rep_data(train_data[c])
    median_rep_data_test[c] = categorical_rep_data(test_data[c])"""

"# constants\nCATEGORICAL_FIELDS = ['PRI_jet_num']\n\nfor c in CATEGORICAL_FIELDS:\n    median_rep_data_train[c] = categorical_rep_data(train_data[c])\n    median_rep_data_test[c] = categorical_rep_data(test_data[c])"

In [67]:
tx_train = delete_features(tx_train, 0.9)
tx_train.shape

(30, 250000)

In [87]:
tx_train, y_train = balance(tx_train.T, y_train, LOWER_BOUND, UPPER_BOUND)

In [88]:
tx_train = replace_nan_by_median(tx_train)
tx_test = replace_nan_by_median(tx_test)

In [69]:
#tx_train = min_max(tx_train)

In [89]:
mean_train, std_train, tx_train = standardize(tx_train)
tx_test = standardize_predef(tx_test, mean_train, std_train)

# Logistic Regression

In [90]:
tx_train
tx_train.shape

(30, 171334)

# Cross Validation

In [91]:
from implementations import build_k_indices
#from tqdm import tqdm_notebook
from matplotlib.pyplot import figure, show
from matplotlib.ticker import MaxNLocator
import matplotlib.patches as mpatches

def cross_validation_demo():
    model = "ridge_regression"
    seed = 3
    k_fold = 10
    degrees = np.arange(9, 12, 1)
    lambdas = np.logspace(-4, -2, 20)
    gammas = [0]#np.arange(0.0, 1.0, 0.05)
    #initial_w = init_w(tx_train)
    max_iters = 200
    k_indices = build_k_indices(y_train, k_fold, seed)
    results = []
    for degree in degrees:
        tx_train_poly = build_poly_tx(tx_train, degree)
        initial_w = init_w(tx_train_poly)
        for gamma in gammas:
            for lambda_ in lambdas:
                accs = []
                ws = []
                for k in range(k_fold):
                    w_tr, acc = cross_validation(y_train, tx_train_poly.T, initial_w,
                                                 max_iters,  k_indices, k, gamma, lambda_, LOWER_BOUND, UPPER_BOUND, model)
                    ws.append(w_tr)
                    accs.append(acc)
                w_mean = np.mean(ws, axis=0)
                acc_mean = np.mean(accs)
                results.append((degree, gamma, lambda_, acc_mean, w_mean))
                
                print("Finished: " + str((degree, gamma, lambda_, acc_mean)))

                '''
                fig = plt.subplots(1, 1, figsize=(10,5))
                plt.plot(range(1,k_fold+1), accs, marker=".", color='b', label='accuracy')
                plt.axhline(y=acc_mean, color='r', label='mean')
                ax = plt.gca()
                ax.xaxis.set_major_locator(MaxNLocator(integer=True))
                plt.xlabel("k")
                plt.ylabel("accuracy")
                plt.title("k-fold accuracy for lambda=%.5f, gamma=%.2f, degree=%d" % (lambda_, gamma, degree))
                plt.legend(loc=2)
                plt.grid(True)
                plt.show()
                '''
    
    return results

results = cross_validation_demo()

Finished: (9, 0, 0.0001, 0.800169263993463)
Finished: (9, 0, 0.00012742749857031334, 0.80015759061460334)
Finished: (9, 0, 0.0001623776739188721, 0.80010506040973561)
Finished: (9, 0, 0.00020691380811147902, 0.79997081655285129)
Finished: (9, 0, 0.00026366508987303583, 0.79987742952197516)
Finished: (9, 0, 0.00033598182862837811, 0.79989493959026436)
Finished: (9, 0, 0.00042813323987193956, 0.79977820580166914)
Finished: (9, 0, 0.00054555947811685203, 0.79966730870250402)
Finished: (9, 0, 0.00069519279617756048, 0.79966730870250391)
Finished: (9, 0, 0.00088586679041008235, 0.79949220801961129)
Finished: (9, 0, 0.0011288378916846883, 0.79921788361641277)
Finished: (9, 0, 0.0014384498882876629, 0.79875678515146198)
Finished: (9, 0, 0.0018329807108324356, 0.79769450767524663)
Finished: (9, 0, 0.0023357214690901212, 0.79472363275550106)
Finished: (9, 0, 0.0029763514416313191, 0.77449950388139865)
Finished: (9, 0, 0.0037926901907322499, 0.79923539368470209)
Finished: (9, 0, 0.00483293023857

In [95]:
results.sort(key=lambda x: -x[3])
[x[0] for x in results[:4]]

[11, 11, 11, 11]

In [96]:
results.sort(key=lambda x: -x[3])
best_ws = [x[4] for x in results[:4]]
mean_w = np.mean(best_ws, axis=0)

tx_test_poly = build_poly_tx(tx_test, 11)
tx_test_poly.shape

(331, 568238)

# Predict labels

In [97]:
from proj1_helpers import predict_labels_kaggle

In [98]:
y_pred = predict_labels_kaggle(mean_w, tx_test_poly.T, LOWER_BOUND, UPPER_BOUND)

# Submission

In [99]:
from proj1_helpers import create_csv_submission

In [100]:
create_csv_submission(ids_test, y_pred, "test14.csv")

Test n : algorithm / features / y / w
- - - - - - - - - - - - - - - - - - - 
Test 1 : least_squares / all features standardized / y = -1,1 / random init_w

Test 2 : least_squares / corr > 0.1 features standardized / y = -1,1 / random init_w

Test 3 : least_squares_GD(10000,0.5) / all features standardized / y = -1,1 / random init_w

Test 4 : least_squares_GD(10000,0.5) / all features standardized / y = -1,1 / random init_w / poly, degree=1

Test 5 : least_squares / all features standardized / y = -1,1 / random init_w / median + categorical

Test 6 : logistic_regressoin  /all features standardized / y = 0,1 / random init_w / median + categorical + balanced

Test 7 : least_squares / all features standardized / y = 0,1 / random init_w

Test 8 : Test 1

Test 9 : Test 1

Test 10 : Test 1 / standardized test_set with mean and std from train_set

Test 11 : Test 1 / standardized test_set with mean and std from train_set / balance

Test 12 : Ridge regression / non-balanced / standardized test_set with mean and std from train_set / y=-1,1 / mean of 4 best lambdas for degree 11

Test 13 : Ridge regression / balanced / standardized test_set with mean and std from train_set / y=-1,1 / mean of 4 best lambdas for degree 11

Test 13 : Ridge regression / balanced before doing anything / standardized test_set with mean and std from train_set / y=-1,1 / mean of 4 best lambdas for degree 11

## Further work

- balance output (batch numpy)
- median and category
- features engineering : features d'intéraction
- logistic regression 